## This is a short tutorial on explaining the difference between Automatic Differentiation and Symbolic Differentiation

This material is intended for readers who have read (and not yet understood) the forward/reverse modes of Automatic Differentiation. In the notebook, there is some original experimentation, along with compiled materials from  [Automatic Differentiation in Machine Learning](https://arxiv.org/abs/1502.05767) and various other sources. 

It is said that AD tools are one of the most underrated and under-utilized tools in today's ML. What makes ADs so special? 

While it is ease to see the difference between Automatic Differentiation (AD) vs Numerical Differentiation (ND)  (which is marred by approximation errors), the subtle difference between AD vs Symbolic Differentiation (SDs) is not easy to appreciate. 

The key idea is this: In symoblic differentiation, we first regliously evalute the complete expression and then differentiate a complex expression using sum rule, product rule, etc. However, it may so happen that we are indeed repeteadly evaluating a same expression multiple times. For e.g.

$\begin{equation}
\begin{aligned}
\frac{d}{dx} \left(f(x) + g(x)\right) &\leadsto \frac{d}{dx} f(x) + \frac{d}{dx} g(x)\\
\frac{d}{dx} \left(f(x)\,g(x)\right) &\leadsto \left(\frac{d}{dx} f(x)\right) g(x) + f(x) \left(\frac{d}{dx} g(x)\right)\; .
\end{aligned}
\label{EquationMultiplicationRule}
\end{equation}$


Consider a function $h(x)=f(x)g(x)$ and the multiplication rule in the above equation. Since $h$ is a product, $h(x)$ and $\frac{d}{dx}h(x)$ have some common components, namely $f(x)$ and $g(x)$. Note also that on the right hand side, $f(x)$ and $\frac{d}{dx}f(x)$ appear separately. If we just proceeded to symbolically differentiate $f(x)$ and plugged its derivative into the appropriate place, we would have nested duplications of any computation that appears in common between $f(x)$ and $\frac{d}{dx}f(x)$. Hence, careless symbolic differentiation can easily produce exponentially large symbolic expressions which take correspondingly long to evaluate. This problem is known as **expression swell**.


When we are concerned with the accurate numerical evaluation of derivatives and not so much with their actual symbolic form, it is in principle possible to significantly simplify computations by storing only the values of intermediate sub-expressions in memory. Moreover, for further efficiency, we can interleave as much as possible the differentiation and simplification steps. This interleaving idea forms the basis of AD and provides an account of its simplest form: **apply symbolic differentiation at the elementary operation level and keep intermediate numerical results, in lockstep with the evaluation of the main function.** 

To illustrate the problem, consider the following example: Iterations of the logistic map $l_{n+1}=4l_n (1-l_n)$, $l_1=x$ and the corresponding derivatives of $l_n$ with respect to $x$, illustrating expression swell.

\begin{array}{rr} \hline
n & l_n & \frac{d}{dx}l_n & \frac{d}{dx}l_n (Simplified form) \\ \hline
1 & x & 1 & 1 \\ \hline
2 & 4x(1 - x) & 4(1 - x) -4x & 4 - 8x \\ \hline
3 & 16x(1 - x)(1 - 2 x)^2 & 16(1 - x)(1 - 2 x)^2 - 16x(1 - 2 x)^2 - 64x(1 - x)(1 - 2 x) & 16 (1 - 10 x + 24 x^2 - 16 x^3) \\
4 & 64x(1 - x)(1 - 2 x)^2 (1 - 8 x + 8 x^2)^2 & 128x(1 - x)(-8 + 16 x)(1 - 2 x)^2 (1 - 8 x + 8 x^2) + 64 (1 - x)(1 - 2 x)^2  (1 - 8 x + 8 x^2)^2 - 64x(1 - 2 x)^2 (1 - 8 x + 8 x^2)^2 - 256x(1 - x)(1 - 2 x)(1 - 8 x + 8 x^2)^2 & 64 (1 - 42 x + 504 x^2 - 2640 x^3 + 7040 x^4 - 9984 x^5 + 7168 x^6 - 2048 x^7)\\
\end{array}

The above table clearly shows that the number of repetitive evaluations increase with $n$.

**A rudimentary solution to counter expression swell:**

For e.g.: $x(1-x)$ occurs many times in the derivative. So, while computing the $\frac{dl}{dx}$ say at $x = 0.5$, we can compute this value of $x(1-x) = 0.5*0.5 = 0.25$ once and use it whenever we encounter. 

**What AD does actually**
Rather than first getting the expression of $l_{n+1}$ entirely in terms of x and then differentiating, we can do the following: 
The derivative of $l_{n+1} = 4 l_{n} (1- l_{n})$ can be found using the chain rule $\frac{dl_{n+1}}{dl_n}\frac{dl_{n}}{dl_{n-1}}\dots \frac{dl_{1}}{dl_x}$ which simplfies to $4(1-l_{n} - l_{n})4(1-l_{n-1} - l_{n-1})\dots4(1-x - x)$. 

The thing to note is that evaluating the AD way is computationally linear wrt $n$ (because we add only one $(1-l_n - l_n)$ for each increase by 1) whereas using the SD way is exponential (as evident in the table above). This linear timecomplexity is achieved due to carry-over of the derivatives at each step, rather than evaluating the derivative at the end and subsituting the value of x. This is the crux of AD. Of course, there are other important differences between AD vs SD but they are not considered here. 


#### An example illustrating the difference due to AD vs SD on time complexity

Consider the following expressions: 
$l_0 = \frac{1}{1+e^x}$, $l_1 = \frac{1}{1+e^{l_0}} $, ....., $l_{n} = \frac{1}{1+e^{l_{n-1}}} $

We evaluate the derivative of $l_{n}$ wrt x and compare the runtime in Mathematica (SD) vs Python (AD) for various values of $n$

In [4]:
## Python Code
import autograd.numpy as np
from autograd import grad
import time

def my_sin(x,n):
    p = x
    y = 1 / (1+np.exp(p))
    for i in range(n):
        y = 1 /(1+ np.exp(y))
        #y = (1+ np.exp(y))
    return y
grad_sin = grad(my_sin)

#computes the mean runtime over 1000 iterations

def my_timer(x):
    start_time = time.clock()
    times = []
    for i in range(1000):
        start_time = time.clock()
        grad_sin(0.0,x)
        times.append(time.clock() - start_time)
    return np.mean(times)    

# printing the runtime (in seconds) for values of n
print(my_timer(1))
print(my_timer(5))
print(my_timer(10))
print(my_timer(50))
print(my_timer(100))
print(my_timer(200))

C:\Users\e0335901\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\e0335901\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\e0335901\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


0.00013198895001823984
0.0002991872100128603
0.0005153054840075129
0.0022935320530004903
0.004333592132985359
0.00917335605399603


###### Note that the runtime here is almost linear wrt to n

The equivalent code in Mathematica: 
(for 10 iterations)
f[x_] = Nest[1/(1 + Exp[#]) &, x, 10]
Timing[D[f[x], x] /. x -> 0.0]

$\text{Timing}\left[\frac{\partial f(x)}{\partial x}\text{/.}\, x\to 0.\right] \left(f(\text{x$\_$})=\text{Nest}\left[\frac{1}{\exp (\text{$\#$1})+1}\&,x,10\right]\right)$

The timings (in seconds) obtained are as follows:


In [ ]:
0.000000 
0.000046875 
0.000231662
0.00437423
0.15625
1.45364



\begin{array}{rr} 
 \text{Average} & \text{Runtime over} & \text{ 1000 iterations} &  \\ \hline
n & Python  & Mathematica \\ \hline
1 & 0.0013 & 0.0000 \\ 
5 & 0.000299 & 0.000046 \\ 
10 & 0.000515 & 0.000231 \\
50 & 0.002935 & 0.004374 \\
100 & 0.004333 & 0.15625 \\
200 & 0.0091733 & 1.45364 \\
\end{array}


Note the linear time complexity of Python's Autograd vs Mathematica's exponential time complexity